In [71]:
sc.version


'2.3.3'

PART-1:  "WORDCOUNT" EXAMPLE USING MAPREDUCE

In [137]:
# Fetch the text file for wordcount example
!wget https://raw.githubusercontent.com/ibarabasi/wordcount/master/wordcount
!cat wordcount

--2019-10-22 00:58:00--  https://raw.githubusercontent.com/ibarabasi/wordcount/master/wordcount
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 710 [text/plain]
Saving to: 'wordcount'

100%[======================================>] 710         --.-K/s   in 0s      

2019-10-22 00:58:00 (38.9 MB/s) - 'wordcount' saved [710/710]

Big data refers to the massive amount of data which cannot be stored, processed and analyzed using traditional ways.
The main elements of big data are:
Volume - There is a massive amount of data generated every second.
Velocity - The speed at which data is generated, collected and analyzed
Variety - The different types of data: structured, semi-structured, unstructured
Value - The ability to turn data into useful insights for your business
Veracity - Trustworthiness in terms of

In [142]:
#Simple example to read text file
rdd0 = sc.textFile("wordcount")
rdd0.take(20)

['Big data refers to the massive amount of data which cannot be stored, processed and analyzed using traditional ways.',
 'The main elements of big data are:',
 'Volume - There is a massive amount of data generated every second.',
 'Velocity - The speed at which data is generated, collected and analyzed',
 'Variety - The different types of data: structured, semi-structured, unstructured',
 'Value - The ability to turn data into useful insights for your business',
 'Veracity - Trustworthiness in terms of quality and accuracy',
 'The main challenges that big data faced and the solutions for each are listed below:',
 'Single central storage',
 'Serial processing',
 'One input',
 'One Output',
 'One Processor',
 'Lack of ability to process unstructured data']

In [143]:
word_counts = rdd0.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: (a + b)).map(lambda x:(x[1],x[0]))

word_counts.take(30)


[(1, 'refers'),
 (2, 'massive'),
 (2, 'amount'),
 (6, 'of'),
 (1, 'cannot'),
 (1, 'processed'),
 (1, 'using'),
 (1, 'traditional'),
 (5, 'The'),
 (1, 'are:'),
 (2, 'is'),
 (1, 'generated'),
 (1, 'second.'),
 (1, 'Velocity'),
 (1, 'speed'),
 (1, 'at'),
 (1, 'generated,'),
 (1, 'different'),
 (1, 'semi-structured,'),
 (1, 'Value'),
 (1, 'turn'),
 (1, 'into'),
 (1, 'useful'),
 (1, 'business'),
 (1, 'Veracity'),
 (1, 'in'),
 (1, 'quality'),
 (1, 'challenges'),
 (1, 'faced'),
 (1, 'are')]

PART-2:  "COMMON FRIENDS" EXAMPLE USINg MAPREDUCE

In [144]:
# Load data from github
!wget "https://raw.githubusercontent.com/ibarabasi/wordcount/master/friends"
rdd = sc.textFile("friends")
!cat friends

--2019-10-22 01:00:47--  https://raw.githubusercontent.com/ibarabasi/wordcount/master/friends
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 193 [text/plain]
Saving to: 'friends.2'

100%[======================================>] 193         --.-K/s   in 0s      

2019-10-22 01:00:47 (24.1 MB/s) - 'friends.2' saved [193/193]

me Alice
Henry me
Henry Alice
me Jane
Alice John
Jane John
Judy Alice
me Mary
Mary Joyce
Joyce Henry
Judy me
Judy Jane
John Carol
Carol me
Mary Henry
Louise Ronald
Ronald Thomas
William Thomas


In [109]:
!cat wordcount/friends
# Build the first pair RDD
rdd0 = rdd.map(lambda x: x.split())
rdd0.take(20)

me Alice
Henry me
Henry Alice
me Jane
Alice John
Jane John
Judy Alice
me Mary
Mary Joyce
Joyce Henry
Judy me
Judy Jane
John Carol
Carol me
Mary Henry
Louise Ronald
Ronald Thomas
William Thomas


[['me', 'Alice'],
 ['Henry', 'me'],
 ['Henry', 'Alice'],
 ['me', 'Jane'],
 ['Alice', 'John'],
 ['Jane', 'John'],
 ['Judy', 'Alice'],
 ['me', 'Mary'],
 ['Mary', 'Joyce'],
 ['Joyce', 'Henry'],
 ['Judy', 'me'],
 ['Judy', 'Jane'],
 ['John', 'Carol'],
 ['Carol', 'me'],
 ['Mary', 'Henry'],
 ['Louise', 'Ronald'],
 ['Ronald', 'Thomas'],
 ['William', 'Thomas']]

In [110]:
rdd1=rdd0.union(rdd.map(lambda x: x.split()[::-1]))
# Bring my friend list to local
lst = rdd1.filter(lambda x: x[0] == 'me').map(lambda x: x[1]).collect()
# Build the second pair RDD
rdd2 = rdd1.filter(lambda x: x[0] in lst).map(lambda x: x[1]). \
    filter(lambda x: x != 'me' and x not in lst). \
    map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b). \
    map(lambda x: (x[1], x[0])).sortByKey(ascending = False)
# Bring the result to local since the sample is small
for x, y in rdd2.collect():
    print ("The stranger {} has {} common friends with me".format(y, x))


The stranger John has 3 common friends with me
The stranger Joyce has 2 common friends with me
